# Amex default rate prediction - Kaggle project
## Vidya Shalini Bulusu

### ABOUT ### 
My independent study project is about implementing Machine Learning algorithms using Spark-ML and AWS.

### OBJECTIVE ### 
The Objective of this project is to predict the probability that a customer does not pay back their credit card balance amount in the future based on their monthly customer profile.

How do we determine default event?
- For each customer, we observe 18 months performance window after the latest credit card statement, and if the customer does not pay the due amount in 120 days after their latest statement date it is considered a default event.

### MOTIVATION ###
- to learn and gain experience working with real-life implementation of the skills learnt in the Big Data class, using Spark-ML.



In [2]:
#  # sc.list_packages()
# sc.install_pypi_package("Cython")
# sc.install_pypi_package("pandas==0.25.1")

#  # Install matplotlib from given PyPI repository
# sc.install_pypi_package("matplotlib==3.1.1", "https://pypi.org/simple") 


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import pyspark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation, Summarizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, Imputer, StringIndexer, StandardScaler, PCA
from pyspark.sql.functions import col, isnan, when, count
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Suppress warnings

In [4]:
import warnings
warnings.filterwarnings('ignore')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Get Data and Data Exploration

In [5]:
# path = 'Sample_data.csv' 
# path = "s3://amexdefaultpredictionbucket/data/sample.csv"
path = "s3://amexdefaultpredictionbucket/data/train_data.csv" # Big dataset 15+GB
data = spark.read.csv(path, header=True, inferSchema='true')

# for long rows, set vertical=True 
data.show(1, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0---------------------------
 customer_ID | 0000099d6bd597052... 
 S_2         | 2017-03-09 00:00:00  
 P_2         | 0.9384687191272548   
 D_39        | 0.0017333390041739   
 B_1         | 0.0087244509498605   
 B_2         | 1.0068382339663076   
 R_1         | 0.0092277222786689   
 S_3         | 0.1240351558143743   
 D_41        | 0.0087711319938824   
 B_3         | 0.0047092406313857   
 D_42        | null                 
 D_43        | null                 
 D_44        | 6.30134804911503E-4  
 B_4         | 0.0809863324662527   
 D_45        | 0.708906305121159    
 B_5         | 0.1706002293387026   
 R_2         | 0.0062040314303209   
 D_46        | 0.3585865793715965   
 D_47        | 0.525351040810055    
 D_48        | 0.255736073902975    
 D_49        | null                 
 B_6         | 0.0639022133803909   
 B_7         | 0.0594157330614109   
 B_8         | 0.006465576798311    
 D_50        | 0.1486979558199995   
 D_51        | 1.3358557940752642   
 

#### Load the test dataset

In [6]:
final_test = spark.read.csv("s3://amexdefaultpredictionbucket/data/test_data.csv", 
                            header=True, inferSchema='true')
final_test.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(customer_ID='00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7', S_2=datetime.datetime(2019, 2, 19, 0, 0), P_2=0.631315348870127, D_39=0.0019117764764965, B_1=0.0107280865415314, B_2=0.8144968588558597, R_1=0.0075468996994418, S_3=0.1686508406762685, D_41=0.0099714525530773, B_3=0.0023469930565875, D_42=0.1131888918343083, D_43=None, D_44=0.0075839407750031, B_4=0.1164022289247906, D_45=0.0044825219725134, B_5=0.0095167747593996, R_2=0.0051778780931715, D_46=None, D_47=0.4743833427304856, D_48=0.626466914148911, D_49=None, B_6=0.1745910796093573, B_7=1.2506765348178908, B_8=1.003925464255982, D_50=None, D_51=0.0087178835533091, B_9=0.0010129373793993, R_3=0.1035897430477255, D_52=0.119043469590424, P_3=None, B_10=-0.0029186420127821, D_53=None, S_5=0.0027711123379222, B_11=0.004416113504079, S_6=1.006874529728068, D_54=1.00506876306948, R_4=0.0028096914463902, S_7=0.1055866024109917, B_12=0.1191980794718266, S_8=0.3216764047819389, D_55=0.114563534989097, D_56=None,

In [7]:
len(final_test.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

190

In [8]:
'c_0' in data.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False

In [9]:
pandas_description = data.describe().toPandas()
pandas_description

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  summary  ...                 D_145
0   count  ...               5429903
1    mean  ...   0.06233496140897697
2  stddev  ...   0.19349367960733202
3     min  ...  1.226024014533422E-9
4     max  ...      4.82762988488277

[5 rows x 190 columns]

#### These columns are categorical variables

Spark-dataFrames does not have value_counts method to look at the different values of the categorical variable.

In [10]:
categorical_variables = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
pandas_description[categorical_variables].T

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

             0                     1                    2     3    4
B_30   5529435   0.15798883611074188   0.3909676887340086   0.0  2.0
B_38   5529435    2.7167043287424484   1.5813224702692543   1.0  7.0
D_114  5354735    0.6193542724336498   0.4855456741431727   0.0  1.0
D_116  5354735  0.001237409507660043  0.03515506444555979   0.0  1.0
D_117  5354735     2.319056498594235   2.2887299150170524  -1.0  6.0
D_120  5354735   0.11672136903133395  0.32108801020328914   0.0  1.0
D_126  5414635    0.7390186042087786   0.5378083997080243  -1.0  1.0
D_63   5531451                  None                 None    CL   XZ
D_64   5314009                  -1.0                  0.0    -1    U
D_66    623354    0.9899126339126724  0.09992810991877812   0.0  1.0
D_68   5314948     5.012029844882772     1.35133908848903   0.0  6.0

#### Description
- Column S_2 is timestamp values and has dates.
- Column customer_ID stores the IDs of each customer


Transform 'D_63' and 'D_64' columns from string to int using StringIndexer and Rename new column to old

In [11]:
data.select('D_63').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|D_63|
+----+
|  XL|
|  CL|
|  XZ|
|  XM|
|  CO|
|  CR|
+----+

In [25]:
indexer = StringIndexer(inputCol="D_63", outputCol="D_63_new")
D_63_indexed_Model = indexer.fit(data)
indexed = D_63_indexed_Model.transform(data)

data_indexed = indexed.drop('D_63')
data_indexed = data_indexed.withColumnRenamed('D_63_new', 'D_63')
data_indexed.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(customer_ID='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a', S_2=datetime.datetime(2017, 3, 9, 0, 0), P_2=0.9384687191272548, D_39=0.0017333390041739, B_1=0.0087244509498605, B_2=1.0068382339663076, R_1=0.0092277222786689, S_3=0.1240351558143743, D_41=0.0087711319938824, B_3=0.0047092406313857, D_42=None, D_43=None, D_44=0.000630134804911503, B_4=0.0809863324662527, D_45=0.708906305121159, B_5=0.1706002293387026, R_2=0.0062040314303209, D_46=0.3585865793715965, D_47=0.525351040810055, D_48=0.255736073902975, D_49=None, B_6=0.0639022133803909, B_7=0.0594157330614109, B_8=0.006465576798311, D_50=0.1486979558199995, D_51=1.3358557940752642, B_9=0.0082067391252636, R_3=0.0014225021561254, D_52=0.2073338786110817, P_3=0.7364627260945562, B_10=0.0962188066642168, D_53=None, S_5=0.0233811220509184, B_11=0.0027680616648439, S_6=0.0083216461012018, D_54=1.0015189798143298, R_4=0.0082984353659013, S_7=0.1613448213392926, B_12=0.1482660660500143, S_8=0.9229980580065352, D_5

In [26]:
indexed.select('D_63').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|D_63|
+----+
|  XL|
|  CL|
|  XZ|
|  XM|
|  CO|
|  CR|
+----+

In [27]:
indexed_test = D_63_indexed_Model.transform(final_test)

data_indexed_test = indexed_test.drop('D_63')
data_indexed_test = data_indexed_test.withColumnRenamed('D_63_new', 'D_63')
data_indexed_test.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(customer_ID='00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7', S_2=datetime.datetime(2019, 2, 19, 0, 0), P_2=0.631315348870127, D_39=0.0019117764764965, B_1=0.0107280865415314, B_2=0.8144968588558597, R_1=0.0075468996994418, S_3=0.1686508406762685, D_41=0.0099714525530773, B_3=0.0023469930565875, D_42=0.1131888918343083, D_43=None, D_44=0.0075839407750031, B_4=0.1164022289247906, D_45=0.0044825219725134, B_5=0.0095167747593996, R_2=0.0051778780931715, D_46=None, D_47=0.4743833427304856, D_48=0.626466914148911, D_49=None, B_6=0.1745910796093573, B_7=1.2506765348178908, B_8=1.003925464255982, D_50=None, D_51=0.0087178835533091, B_9=0.0010129373793993, R_3=0.1035897430477255, D_52=0.119043469590424, P_3=None, B_10=-0.0029186420127821, D_53=None, S_5=0.0027711123379222, B_11=0.004416113504079, S_6=1.006874529728068, D_54=1.00506876306948, R_4=0.0028096914463902, S_7=0.1055866024109917, B_12=0.1191980794718266, S_8=0.3216764047819389, D_55=0.114563534989097, D_56=None, 

Since we don't know what the values in D_64 represent, lets fillna with -2 to handle the 'null' values without affecting the existing classes.

In [28]:
'D_64' in data_indexed.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [29]:
data_indexed.select('D_64').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|D_64|
+----+
|null|
|   U|
|   R|
|   O|
|  -1|
+----+

In [30]:
data_indexed = data_indexed.fillna({'D_64':-2})
data_indexed.select('D_64').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|D_64|
+----+
|   U|
|   R|
|  -2|
|   O|
|  -1|
+----+

In [31]:
indexer = StringIndexer(inputCol="D_64", outputCol="D_64_new")
D_64_indexed_Model = indexer.fit(data_indexed)
indexed = D_64_indexed_Model.transform(data_indexed)

data_indexed = indexed.drop('D_64')
data_indexed = data_indexed.withColumnRenamed('D_64_new', 'D_64')
data_indexed.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(customer_ID='0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a', S_2=datetime.datetime(2017, 3, 9, 0, 0), P_2=0.9384687191272548, D_39=0.0017333390041739, B_1=0.0087244509498605, B_2=1.0068382339663076, R_1=0.0092277222786689, S_3=0.1240351558143743, D_41=0.0087711319938824, B_3=0.0047092406313857, D_42=None, D_43=None, D_44=0.000630134804911503, B_4=0.0809863324662527, D_45=0.708906305121159, B_5=0.1706002293387026, R_2=0.0062040314303209, D_46=0.3585865793715965, D_47=0.525351040810055, D_48=0.255736073902975, D_49=None, B_6=0.0639022133803909, B_7=0.0594157330614109, B_8=0.006465576798311, D_50=0.1486979558199995, D_51=1.3358557940752642, B_9=0.0082067391252636, R_3=0.0014225021561254, D_52=0.2073338786110817, P_3=0.7364627260945562, B_10=0.0962188066642168, D_53=None, S_5=0.0233811220509184, B_11=0.0027680616648439, S_6=0.0083216461012018, D_54=1.0015189798143298, R_4=0.0082984353659013, S_7=0.1613448213392926, B_12=0.1482660660500143, S_8=0.9229980580065352, D_

In [32]:
data_indexed_test = data_indexed_test.fillna({'D_64':-2})
data_indexed_test.select('D_64').distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|D_64|
+----+
|   U|
|   R|
|  -2|
|   O|
+----+

In [33]:
indexed_test = D_64_indexed_Model.transform(data_indexed_test)

data_indexed_test = indexed_test.drop('D_64')
data_indexed_test = data_indexed_test.withColumnRenamed('D_64_new', 'D_64')
data_indexed_test.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(customer_ID='00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7', S_2=datetime.datetime(2019, 2, 19, 0, 0), P_2=0.631315348870127, D_39=0.0019117764764965, B_1=0.0107280865415314, B_2=0.8144968588558597, R_1=0.0075468996994418, S_3=0.1686508406762685, D_41=0.0099714525530773, B_3=0.0023469930565875, D_42=0.1131888918343083, D_43=None, D_44=0.0075839407750031, B_4=0.1164022289247906, D_45=0.0044825219725134, B_5=0.0095167747593996, R_2=0.0051778780931715, D_46=None, D_47=0.4743833427304856, D_48=0.626466914148911, D_49=None, B_6=0.1745910796093573, B_7=1.2506765348178908, B_8=1.003925464255982, D_50=None, D_51=0.0087178835533091, B_9=0.0010129373793993, R_3=0.1035897430477255, D_52=0.119043469590424, P_3=None, B_10=-0.0029186420127821, D_53=None, S_5=0.0027711123379222, B_11=0.004416113504079, S_6=1.006874529728068, D_54=1.00506876306948, R_4=0.0028096914463902, S_7=0.1055866024109917, B_12=0.1191980794718266, S_8=0.3216764047819389, D_55=0.114563534989097, D_56=None,

### Counting the rows per Customer for 10 customers

In [34]:
data_indexed.groupBy('customer_ID').count().orderBy('count', ascending=False).head(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(customer_ID='027c8767b49e4cc8fd28f99e34abbc7c1128281874b7446031af73a1d49426ab', count=13), Row(customer_ID='027e587f6c9af10cacbe41d967be389df40660a163156c5a682262993461747c', count=13), Row(customer_ID='027dd06ce3ad37e17b65054de6a436e18b5b31671e5ed0bd1cf567b8a8f3cd8b', count=13), Row(customer_ID='02e80169c154151914081dea83d80d0719260ecc0df3e2b1ce519a7dddc76d27', count=13), Row(customer_ID='02f63a13e047d26129c5c8334153fe15d7dd406173c486238c1fc38ff3936426', count=13), Row(customer_ID='03c15c67d1b9f6d65aacfd43ddd1f36b5e6a7a6e9d4ba216edd9f2111259be06', count=13), Row(customer_ID='0340d2d354934b2fda3289baaa3228269aa391c4e169f6ce06025b9a9ec2721d', count=13), Row(customer_ID='036e7905324f07948592b09114ca0c23270160e90b9ee8db1cb12db0f66fe5e3', count=13), Row(customer_ID='04212748c5e9354ca364e9e87e884c4af56494b7e98a435f76b879e04737d726', count=13), Row(customer_ID='030c9c701aa2f2c4222e2b7822443ccb31ab622dea8ce3d2381bd980fd961fe4', count=13)]

In [35]:
len(data_indexed.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

190

#### Example of how data is organized per record per customer

In [36]:
data_indexed[data_indexed.customer_ID == '002b4831b4dbff67f987b2818b9d7d1075f6e6e7e32bde8638d6ce0fc2f4eb36'].toPandas()[['S_2','P_2','D_39', 'B_1', 'B_2']]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          S_2       P_2      D_39       B_1       B_2
0  2017-03-18  0.609911  0.030856  0.160618  0.879868
1  2017-04-17  0.549018  0.002802  0.002606  1.000623
2  2017-05-18  0.701254  0.004628  0.009557  0.811009
3  2017-06-17  0.659206  0.002200  0.000336  0.815992
4  2017-07-18  0.605519  0.005084  0.000931  0.812954
5  2017-08-18  0.642984  0.003526  0.007709  0.812181
6  2017-09-16  0.689937  0.006455  0.007071  0.815515
7  2017-10-18  0.665456  0.009472  0.008935  0.813391
8  2017-11-17  0.628402  0.009246  0.009333  0.811796
9  2017-12-18  0.624550  0.005651  0.004205  0.819339
10 2018-01-18  0.632544  0.007562  0.006856  0.814759
11 2018-02-15  0.631396  0.001678  0.008010  0.814094
12 2018-03-15  0.600478  0.003848  0.005332  0.815046

### Data Cleaning and Dimensionality Reduction

The dataset has 190 columns and has been anonymized and normalized. 
This means that we do not need to normalize the data again.

The next step is to clean up the dataset by:
1. identify columns with more than 80% nulls and remove them
2. there are more columns with over 70% nulls - impute with median values
3. remove columns that are highly correlated to other columns

In [37]:
threshold_to_drop = int(0.8 * data_indexed.count())
threshold_to_drop

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4425160

#### Initially thought I can use dropna to drop columns. But PySpark DataFrames does not have an axis keyword, hence, in this case, rows will be deleted. Since we do not want to lose records, upon further research, I found another method to do this, which can be seen in the next cells.
```
# remove columns with more than 80% null values
data.dropna(thresh=threshold_to_drop) # does not work!
```

#### From the above list of columns, we can count null values using PySpark sql functions

In [38]:
cols_to_exclude = ['S_2', 'customer_ID']
columnList = [col for col in data_indexed.columns if col not in cols_to_exclude]
null_percent_cols = data_indexed.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) 
                                 for c in columnList]).toPandas()/data_indexed.count()
null_percent_cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

        P_2  D_39  B_1       B_2  R_1  ...     D_143     D_144     D_145  D_63  D_64
0  0.008313   0.0  0.0  0.000364  0.0  ...  0.018358  0.007363  0.018358   0.0   0.0

[1 rows x 188 columns]

#### Filtering columns with more than 80% missing values

In [39]:
all_valid_cols = list(null_percent_cols.T[null_percent_cols.T[0] < .8].index.to_numpy())
len(all_valid_cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

165

In [40]:
non_cat_cols = [col for col in all_valid_cols if col not in categorical_variables]
len(non_cat_cols), non_cat_cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(155, ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'P_4', 'D_74', 'D_75', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'S_18', 'D_86', 'R_17', 'R_18', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'D_102', 'D_103', 'D_104', 'D_105', 'D_107', 'B_36', 'B_37'

### Fillna for non-categorical variables only

Imputing using pyspark.sql.GroupedData

In [41]:
# grouped_by_cust = data_indexed.groupBy('customer_ID')
# grouped_by_cust.mean('P_2').take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
# # grouped_by_cust.fillna([grouped_by_cust.mean(col) for col in non_cat_cols])

# from pyspark.sql.functions import pandas_udf, PandasUDFType

# def impute_means(pdf):
#      return pd.DataFrame(pdf.v.mean())
# data.groupby('customer_ID').applyInPandas(impute_means).show() 

# grouped_by_cust.apply(impute_means)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# Get non-int columns 
non_int_cols = [c[0] for c in data_indexed.dtypes if c[1] in ["float", "double", "long"]]
non_int_cols = [c for c in non_int_cols if c not in categorical_variables]
non_int_cols = [c for c in non_int_cols if c in all_valid_cols]
len(non_int_cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

154

### Using Imputer to impute median values

Imputing floats and then categorical variables

In [44]:
imputer = Imputer()
imputer.setInputCols(non_int_cols)
imputer.setOutputCols(["{}_imputed".format(c) for c in non_int_cols]).setStrategy("median")
float_imputer = imputer.fit(data_indexed)
data_imputed_floats = float_imputer.transform(data_indexed)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
data_imputed_floats.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'R_2', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'S_6', 'D_54', 'R_4', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'D_66', 'B_20', 'D_68', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'R_15', 'D_84', 'R_16', 'B_29', 'B_30', 'S_18', 'D_86', 'D_87', 'R_17', 'R_18', 'D_88', 'B_31', 'S_19', 'R_19', 'B_32', 'S_20', 'R_20', 'R_21', 'B_33', 'D_89', 'R_22', 'R_23', 'D_91', 'D_92', 'D_93', 'D_94', 'R_24', 'R_25', 'D_9

In [46]:
data_imputed_floats_test = float_imputer.transform(data_indexed_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
data_imputed_floats_test.head(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(customer_ID='00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7', S_2=datetime.datetime(2019, 2, 19, 0, 0), P_2=0.631315348870127, D_39=0.0019117764764965, B_1=0.0107280865415314, B_2=0.8144968588558597, R_1=0.0075468996994418, S_3=0.1686508406762685, D_41=0.0099714525530773, B_3=0.0023469930565875, D_42=0.1131888918343083, D_43=None, D_44=0.0075839407750031, B_4=0.1164022289247906, D_45=0.0044825219725134, B_5=0.0095167747593996, R_2=0.0051778780931715, D_46=None, D_47=0.4743833427304856, D_48=0.626466914148911, D_49=None, B_6=0.1745910796093573, B_7=1.2506765348178908, B_8=1.003925464255982, D_50=None, D_51=0.0087178835533091, B_9=0.0010129373793993, R_3=0.1035897430477255, D_52=0.119043469590424, P_3=None, B_10=-0.0029186420127821, D_53=None, S_5=0.0027711123379222, B_11=0.004416113504079, S_6=1.006874529728068, D_54=1.00506876306948, R_4=0.0028096914463902, S_7=0.1055866024109917, B_12=0.1191980794718266, S_8=0.3216764047819389, D_55=0.114563534989097, D_56=None,

#### Impute categorical variables using method='median': 'mode' was not available

In [48]:
imputer = Imputer()
imputer.setInputCols(categorical_variables)
imputer.setOutputCols(["{}_imputed".format(c) for c in categorical_variables]).setStrategy("median")

cat_imputer = imputer.fit(data_imputed_floats)
data_imputed = cat_imputer.transform(data_imputed_floats)
data_imputed

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[customer_ID: string, S_2: timestamp, P_2: double, D_39: double, B_1: double, B_2: double, R_1: double, S_3: double, D_41: double, B_3: double, D_42: double, D_43: double, D_44: double, B_4: double, D_45: double, B_5: double, R_2: double, D_46: double, D_47: double, D_48: double, D_49: double, B_6: double, B_7: double, B_8: double, D_50: double, D_51: double, B_9: double, R_3: double, D_52: double, P_3: double, B_10: double, D_53: double, S_5: double, B_11: double, S_6: double, D_54: double, R_4: double, S_7: double, B_12: double, S_8: double, D_55: double, D_56: double, B_13: double, R_5: double, D_58: double, S_9: double, B_14: double, D_59: double, D_60: double, D_61: double, B_15: double, S_11: double, D_62: double, D_65: double, B_16: double, B_17: double, B_18: double, B_19: double, D_66: double, B_20: double, D_68: double, S_12: double, R_6: double, S_13: double, B_21: double, D_69: double, B_22: double, D_70: double, D_71: double, D_72: double, S_15: double, B_23: doub

In [49]:
len(data_imputed.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

355

In [50]:
len(data_imputed[["{}_imputed".format(c) for c in non_int_cols + categorical_variables]].columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

165

In [51]:
data_imputed_cats_test = cat_imputer.transform(data_imputed_floats_test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
data_imputed_cats_test.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(customer_ID='00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7', S_2=datetime.datetime(2019, 2, 19, 0, 0), P_2=0.631315348870127, D_39=0.0019117764764965, B_1=0.0107280865415314, B_2=0.8144968588558597, R_1=0.0075468996994418, S_3=0.1686508406762685, D_41=0.0099714525530773, B_3=0.0023469930565875, D_42=0.1131888918343083, D_43=None, D_44=0.0075839407750031, B_4=0.1164022289247906, D_45=0.0044825219725134, B_5=0.0095167747593996, R_2=0.0051778780931715, D_46=None, D_47=0.4743833427304856, D_48=0.626466914148911, D_49=None, B_6=0.1745910796093573, B_7=1.2506765348178908, B_8=1.003925464255982, D_50=None, D_51=0.0087178835533091, B_9=0.0010129373793993, R_3=0.1035897430477255, D_52=0.119043469590424, P_3=None, B_10=-0.0029186420127821, D_53=None, S_5=0.0027711123379222, B_11=0.004416113504079, S_6=1.006874529728068, D_54=1.00506876306948, R_4=0.0028096914463902, S_7=0.1055866024109917, B_12=0.1191980794718266, S_8=0.3216764047819389, D_55=0.114563534989097, D_56=None,

#### List of columns for clean dataframe

In [53]:
# int type columns
int_col = [c[0] for c in data_indexed.dtypes if c[1] in ["int"]]
int_col = [c for c in int_col if c in all_valid_cols]
print(len(int_col))
clean_col_lst = cols_to_exclude + ["{}_imputed".format(c) for c in categorical_variables] + ["{}_imputed".format(c) for c in non_int_cols] + int_col
len(clean_col_lst)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1
168

In [54]:
data_clean = data_imputed[[clean_col_lst]]
data_clean.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['S_2', 'customer_ID', 'B_30_imputed', 'B_38_imputed', 'D_114_imputed', 'D_116_imputed', 'D_117_imputed', 'D_120_imputed', 'D_126_imputed', 'D_63_imputed', 'D_64_imputed', 'D_66_imputed', 'D_68_imputed', 'P_2_imputed', 'D_39_imputed', 'B_1_imputed', 'B_2_imputed', 'R_1_imputed', 'S_3_imputed', 'D_41_imputed', 'B_3_imputed', 'D_43_imputed', 'D_44_imputed', 'B_4_imputed', 'D_45_imputed', 'B_5_imputed', 'R_2_imputed', 'D_46_imputed', 'D_47_imputed', 'D_48_imputed', 'B_6_imputed', 'B_7_imputed', 'B_8_imputed', 'D_50_imputed', 'D_51_imputed', 'B_9_imputed', 'R_3_imputed', 'D_52_imputed', 'P_3_imputed', 'B_10_imputed', 'D_53_imputed', 'S_5_imputed', 'B_11_imputed', 'S_6_imputed', 'D_54_imputed', 'R_4_imputed', 'S_7_imputed', 'B_12_imputed', 'S_8_imputed', 'D_55_imputed', 'D_56_imputed', 'B_13_imputed', 'R_5_imputed', 'D_58_imputed', 'S_9_imputed', 'B_14_imputed', 'D_59_imputed', 'D_60_imputed', 'D_61_imputed', 'B_15_imputed', 'S_11_imputed', 'D_62_imputed', 'D_65_imputed', 'B_16_imputed', 'B

In [55]:
data_clean_test = data_imputed_cats_test[[clean_col_lst]]
data_clean_test.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['S_2', 'customer_ID', 'B_30_imputed', 'B_38_imputed', 'D_114_imputed', 'D_116_imputed', 'D_117_imputed', 'D_120_imputed', 'D_126_imputed', 'D_63_imputed', 'D_64_imputed', 'D_66_imputed', 'D_68_imputed', 'P_2_imputed', 'D_39_imputed', 'B_1_imputed', 'B_2_imputed', 'R_1_imputed', 'S_3_imputed', 'D_41_imputed', 'B_3_imputed', 'D_43_imputed', 'D_44_imputed', 'B_4_imputed', 'D_45_imputed', 'B_5_imputed', 'R_2_imputed', 'D_46_imputed', 'D_47_imputed', 'D_48_imputed', 'B_6_imputed', 'B_7_imputed', 'B_8_imputed', 'D_50_imputed', 'D_51_imputed', 'B_9_imputed', 'R_3_imputed', 'D_52_imputed', 'P_3_imputed', 'B_10_imputed', 'D_53_imputed', 'S_5_imputed', 'B_11_imputed', 'S_6_imputed', 'D_54_imputed', 'R_4_imputed', 'S_7_imputed', 'B_12_imputed', 'S_8_imputed', 'D_55_imputed', 'D_56_imputed', 'B_13_imputed', 'R_5_imputed', 'D_58_imputed', 'S_9_imputed', 'B_14_imputed', 'D_59_imputed', 'D_60_imputed', 'D_61_imputed', 'B_15_imputed', 'S_11_imputed', 'D_62_imputed', 'D_65_imputed', 'B_16_imputed', 'B

### Finding Cross Correlation of columns of type "double"

Pyspark DataFrame does not have pandas equivalent of xcorr method.
But it has Correlation.corr method which takes dataset and feature vectors as input.
(https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.stat.Correlation.html?highlight=correlation)

To convert the columns to Vectors, I used the VectorAssembler method:
``` from pyspark.ml.feature import VectorAssembler ```
It wasn't clear that I had to use this from the errors I was getting. AFter some research, I found an example on StackOverflow.



Cross Correlation for float type columns

In [56]:
# convert to vector column first
imputed_non_int_cols = ["{}_imputed".format(c) for c in non_int_cols]
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=imputed_non_int_cols, outputCol=vector_col)
df_vector = assembler.transform(data_clean).select(vector_col)
df_vector.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['corr_features']

In [57]:
# dataset = spark.createDataFrame(data, ['features'])
correlation = Correlation.corr(df_vector, vector_col, method='spearman')
correlation.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                             spearman(corr_features)
0  DenseMatrix([[ 1.        , -0.1621399 , -0.478...

In [58]:
len(correlation.collect()[0]["spearman({})".format(vector_col)].values)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23716

In [59]:
np.reshape(correlation.collect()[0]["spearman({})".format(vector_col)].values,
           (len(imputed_non_int_cols),len(imputed_non_int_cols)))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 1.        , -0.1621399 , -0.47842691, ..., -0.1213763 ,
        -0.01284764, -0.12476138],
       [-0.1621399 ,  1.        ,  0.42687361, ...,  0.0239593 ,
         0.0076399 ,  0.0239038 ],
       [-0.47842691,  0.42687361,  1.        , ...,  0.09009239,
         0.02901566,  0.09143725],
       ...,
       [-0.1213763 ,  0.0239593 ,  0.09009239, ...,  1.        ,
         0.22758489,  0.42594676],
       [-0.01284764,  0.0076399 ,  0.02901566, ...,  0.22758489,
         1.        ,  0.22375393],
       [-0.12476138,  0.0239038 ,  0.09143725, ...,  0.42594676,
         0.22375393,  1.        ]])

#### Reshaping the correlation matrix into columnList x columnList shape

In [60]:
# Reshaping the correlation matrix into columnList x columnList shape
data_xcorr = pd.DataFrame(np.reshape(correlation.collect()[0]["spearman({})".format
                                                              (vector_col)].values,
                                   (len(imputed_non_int_cols),len(imputed_non_int_cols))))
data_xcorr

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          0         1         2    ...       151       152       153
0    1.000000 -0.162140 -0.478427  ... -0.121376 -0.012848 -0.124761
1   -0.162140  1.000000  0.426874  ...  0.023959  0.007640  0.023904
2   -0.478427  0.426874  1.000000  ...  0.090092  0.029016  0.091437
3    0.489942 -0.112510 -0.546878  ... -0.089589 -0.026620 -0.090728
4   -0.237631  0.080908  0.180530  ...  0.033449  0.006149  0.033569
..        ...       ...       ...  ...       ...       ...       ...
149 -0.033586  0.005618  0.019627  ...  0.053918 -0.005623  0.052767
150 -0.116013  0.023345  0.088016  ...  0.426148  0.239174  0.425157
151 -0.121376  0.023959  0.090092  ...  1.000000  0.227585  0.425947
152 -0.012848  0.007640  0.029016  ...  0.227585  1.000000  0.223754
153 -0.124761  0.023904  0.091437  ...  0.425947  0.223754  1.000000

[154 rows x 154 columns]

### Changing column and index names to column names of double type to make it more readable

In [61]:
data_xcorr.columns = imputed_non_int_cols
data_xcorr.index = imputed_non_int_cols
data_xcorr

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

               P_2_imputed  D_39_imputed  ...  D_144_imputed  D_145_imputed
P_2_imputed       1.000000     -0.162140  ...      -0.012848      -0.124761
D_39_imputed     -0.162140      1.000000  ...       0.007640       0.023904
B_1_imputed      -0.478427      0.426874  ...       0.029016       0.091437
B_2_imputed       0.489942     -0.112510  ...      -0.026620      -0.090728
R_1_imputed      -0.237631      0.080908  ...       0.006149       0.033569
...                    ...           ...  ...            ...            ...
D_140_imputed    -0.033586      0.005618  ...      -0.005623       0.052767
D_141_imputed    -0.116013      0.023345  ...       0.239174       0.425157
D_143_imputed    -0.121376      0.023959  ...       0.227585       0.425947
D_144_imputed    -0.012848      0.007640  ...       1.000000       0.223754
D_145_imputed    -0.124761      0.023904  ...       0.223754       1.000000

[154 rows x 154 columns]

In [62]:
data_xcorr_high = data_xcorr.stack()[data_xcorr.stack() > 0.7 ]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
data_xcorr_high[data_xcorr_high == 1].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

154

In [64]:
df = data_xcorr_high[data_xcorr_high < 1]
list(df.index)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('B_1_imputed', 'B_9_imputed'), ('B_1_imputed', 'B_11_imputed'), ('B_1_imputed', 'B_14_imputed'), ('B_1_imputed', 'B_25_imputed'), ('B_1_imputed', 'B_37_imputed'), ('B_2_imputed', 'B_18_imputed'), ('S_3_imputed', 'S_7_imputed'), ('B_3_imputed', 'B_16_imputed'), ('B_3_imputed', 'B_20_imputed'), ('B_4_imputed', 'B_7_imputed'), ('B_4_imputed', 'D_55_imputed'), ('B_4_imputed', 'D_58_imputed'), ('B_4_imputed', 'B_23_imputed'), ('B_4_imputed', 'D_74_imputed'), ('B_4_imputed', 'D_75_imputed'), ('B_4_imputed', 'B_28_imputed'), ('B_4_imputed', 'B_40_imputed'), ('D_47_imputed', 'D_121_imputed'), ('D_48_imputed', 'D_55_imputed'), ('D_48_imputed', 'D_61_imputed'), ('B_6_imputed', 'B_10_imputed'), ('B_6_imputed', 'B_18_imputed'), ('B_7_imputed', 'B_4_imputed'), ('B_7_imputed', 'D_55_imputed'), ('B_7_imputed', 'D_58_imputed'), ('B_7_imputed', 'B_23_imputed'), ('B_7_imputed', 'D_74_imputed'), ('B_7_imputed', 'D_75_imputed'), ('B_7_imputed', 'B_28_imputed'), ('B_7_imputed', 'B_40_imputed'), ('B_9_imp

#### List of highly correlated variables to eliminate

In [65]:
# list of highly correlated variables to eliminate
cols_lst = []
replace_lst = []
for tup in list(df.index):
    if tup[0] not in cols_lst and tup[0] not in replace_lst:
        cols_lst.append(tup[0])
        if tup[1] not in cols_lst and tup[1] not in replace_lst:
            replace_lst.append(tup[1])
    elif tup[0] in cols_lst and tup[0] not in replace_lst:
        if tup[1] not in cols_lst and tup[1] not in replace_lst:
            replace_lst.append(tup[1])
    
print(len(cols_lst), cols_lst)
print('***')
len(replace_lst), replace_lst

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13 ['B_1_imputed', 'B_2_imputed', 'S_3_imputed', 'B_3_imputed', 'B_4_imputed', 'D_47_imputed', 'D_48_imputed', 'B_6_imputed', 'S_5_imputed', 'B_12_imputed', 'S_22_imputed', 'D_102_imputed', 'D_115_imputed']
***
(27, ['B_9_imputed', 'B_11_imputed', 'B_14_imputed', 'B_25_imputed', 'B_37_imputed', 'B_18_imputed', 'S_7_imputed', 'B_16_imputed', 'B_20_imputed', 'B_7_imputed', 'D_55_imputed', 'D_58_imputed', 'B_23_imputed', 'D_74_imputed', 'D_75_imputed', 'B_28_imputed', 'B_40_imputed', 'D_121_imputed', 'D_61_imputed', 'B_10_imputed', 'S_24_imputed', 'B_13_imputed', 'D_103_imputed', 'D_104_imputed', 'D_107_imputed', 'D_118_imputed', 'D_119_imputed'])

#### From all the highly correlated columns, we have reduced the dimensionality to 16.
We also have to add back the categorical variables and the columns that are not highly correlated to each other.

In [66]:
reduced_columnList = [item for item in imputed_non_int_cols if item not in replace_lst]
# clean_col_lst = columnList + categorical_variables
# len(clean_col_lst), clean_col_lst
len(reduced_columnList)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

127

#### The number of columns of type float and that are not categorical has further reduced to 127 from a previous 154.

In [67]:
cleaned_col_lst = cols_to_exclude + reduced_columnList + ["{}_imputed".format(c) for c in categorical_variables] + int_col
len(cleaned_col_lst)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

141

In [68]:
data_cleaned = data_imputed[[cleaned_col_lst]]
data_cleaned.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['S_2', 'customer_ID', 'P_2_imputed', 'D_39_imputed', 'B_1_imputed', 'B_2_imputed', 'R_1_imputed', 'S_3_imputed', 'D_41_imputed', 'B_3_imputed', 'D_43_imputed', 'D_44_imputed', 'B_4_imputed', 'D_45_imputed', 'B_5_imputed', 'R_2_imputed', 'D_46_imputed', 'D_47_imputed', 'D_48_imputed', 'B_6_imputed', 'B_8_imputed', 'D_50_imputed', 'D_51_imputed', 'R_3_imputed', 'D_52_imputed', 'P_3_imputed', 'D_53_imputed', 'S_5_imputed', 'S_6_imputed', 'D_54_imputed', 'R_4_imputed', 'B_12_imputed', 'S_8_imputed', 'D_56_imputed', 'R_5_imputed', 'S_9_imputed', 'D_59_imputed', 'D_60_imputed', 'B_15_imputed', 'S_11_imputed', 'D_62_imputed', 'D_65_imputed', 'B_17_imputed', 'B_19_imputed', 'S_12_imputed', 'R_6_imputed', 'S_13_imputed', 'B_21_imputed', 'D_69_imputed', 'B_22_imputed', 'D_70_imputed', 'D_71_imputed', 'D_72_imputed', 'S_15_imputed', 'P_4_imputed', 'B_24_imputed', 'R_7_imputed', 'D_77_imputed', 'B_26_imputed', 'D_78_imputed', 'D_79_imputed', 'R_8_imputed', 'S_16_imputed', 'D_80_imputed', 'R_10_im

In [69]:
data_cleaned_test = data_imputed_cats_test[[cleaned_col_lst]]
data_cleaned_test

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[S_2: timestamp, customer_ID: string, P_2_imputed: double, D_39_imputed: double, B_1_imputed: double, B_2_imputed: double, R_1_imputed: double, S_3_imputed: double, D_41_imputed: double, B_3_imputed: double, D_43_imputed: double, D_44_imputed: double, B_4_imputed: double, D_45_imputed: double, B_5_imputed: double, R_2_imputed: double, D_46_imputed: double, D_47_imputed: double, D_48_imputed: double, B_6_imputed: double, B_8_imputed: double, D_50_imputed: double, D_51_imputed: double, R_3_imputed: double, D_52_imputed: double, P_3_imputed: double, D_53_imputed: double, S_5_imputed: double, S_6_imputed: double, D_54_imputed: double, R_4_imputed: double, B_12_imputed: double, S_8_imputed: double, D_56_imputed: double, R_5_imputed: double, S_9_imputed: double, D_59_imputed: double, D_60_imputed: double, B_15_imputed: double, S_11_imputed: double, D_62_imputed: double, D_65_imputed: double, B_17_imputed: double, B_19_imputed: double, S_12_imputed: double, R_6_imputed: double, S_13_

### Load the training labels csv

In [70]:
train_labels = spark.read.csv("s3://amexdefaultpredictionbucket/data/train_labels.csv", 
                              header=True, inferSchema='true')
train_labels.describe().toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  summary  ...              target
0   count  ...              458913
1    mean  ...  0.2589336105100531
2  stddev  ...  0.4380495565491465
3     min  ...                   0
4     max  ...                   1

[5 rows x 3 columns]

In [71]:
train_labels.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

458913

In [72]:
# train_labels.show(10, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Aggregate the records per customer

In the train_labels dataframe, there is target variable per customer.
We have to transform the train data in such a way that we aggregate rows per customer to get their trend in the last 12 months. This will result in one row per customer. We can then join the train_labels dataframe to this dataframe.

In [73]:
# We need avg of numeric columns per customerID
data_numeric_per_customer = data_cleaned[[reduced_columnList + 
                                          ['customer_ID']]].groupBy('customer_ID').avg()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
data_numeric_per_customer_test = data_cleaned_test[[reduced_columnList + 
                                          ['customer_ID']]].groupBy('customer_ID').avg()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Sort the rows per customer in descending order of date column and get the first row which represents the latest record of the customer
NOTE: class pyspark.sql.GroupedData does not have .last()

In [75]:
from pyspark.sql import Window
import pyspark.sql.functions as psf
from pyspark.sql.functions import desc

w1 = Window.partitionBy("customer_ID").orderBy(desc("S_2"))
# df1 = df.withColumn("subgroup", psf.sum("newcust").over(w1))
# w2 = Window.partitionBy("customer_ID", "subgroup")
# df2 = df1.withColumn("first_date", psf.min("S_2").over(w2))
# w3 = Window.partitionBy("customer_ID", "subgroup").orderBy(psf.desc("S_2"))
data_categ_per_customer = data_cleaned[[["{}_imputed".format(c) 
                                         for c in categorical_variables] + 
                                        int_col + 
                                        ['customer_ID', 'S_2']]].withColumn(
    "rn", psf.row_number().over(w1)).filter("rn = 1").select(
    ["{}_imputed".format(c) for c in categorical_variables] + int_col + 
    ['customer_ID', 'S_2'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
data_categ_per_customer.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(B_30_imputed=1.0, B_38_imputed=7.0, D_114_imputed=0.0, D_116_imputed=0.0, D_117_imputed=2.0, D_120_imputed=0.0, D_126_imputed=1.0, D_63_imputed=0.0, D_64_imputed=1.0, D_66_imputed=1.0, D_68_imputed=3.0, B_31=1, customer_ID='000473eb907b57c8c23f652bba40f87fe7261273dda47034d46fc46821017e50', S_2=datetime.datetime(2018, 3, 8, 0, 0))]

In [77]:
w1_test = Window.partitionBy("customer_ID").orderBy(desc("S_2"))
data_categ_per_customer_test = data_cleaned_test[[["{}_imputed".format(c) 
                                         for c in categorical_variables] + 
                                        int_col + 
                                        ['customer_ID', 'S_2']]].withColumn(
    "rn", 
    psf.row_number().over(w1_test)
).filter("rn = 1").select( 
    ["{}_imputed".format(c) for c in categorical_variables] + 
                                        int_col + 
                                        ['customer_ID', 'S_2']
)
data_categ_per_customer_test.take(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(B_30_imputed=0.0, B_38_imputed=3.0, D_114_imputed=1.0, D_116_imputed=0.0, D_117_imputed=2.0, D_120_imputed=0.0, D_126_imputed=1.0, D_63_imputed=0.0, D_64_imputed=0.0, D_66_imputed=1.0, D_68_imputed=6.0, B_31=1, customer_ID='000175a73a8417938a1aa41f87ec9ac5141fe67e5085ffaffa3a1d6048cbbbae', S_2=datetime.datetime(2019, 4, 22, 0, 0))]

### Get single row per customer

In [78]:
data_per_cust = data_numeric_per_customer.join(data_categ_per_customer, on='customer_ID')
data_per_cust = data_per_cust.drop('S_2')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
data_per_cust_test = data_numeric_per_customer_test.join(data_categ_per_customer_test, 
                                                         on='customer_ID')
data_per_cust_test = data_per_cust_test.drop('S_2')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
data_per_cust_test.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['customer_ID', 'avg(P_2_imputed)', 'avg(D_39_imputed)', 'avg(B_1_imputed)', 'avg(B_2_imputed)', 'avg(R_1_imputed)', 'avg(S_3_imputed)', 'avg(D_41_imputed)', 'avg(B_3_imputed)', 'avg(D_43_imputed)', 'avg(D_44_imputed)', 'avg(B_4_imputed)', 'avg(D_45_imputed)', 'avg(B_5_imputed)', 'avg(R_2_imputed)', 'avg(D_46_imputed)', 'avg(D_47_imputed)', 'avg(D_48_imputed)', 'avg(B_6_imputed)', 'avg(B_8_imputed)', 'avg(D_50_imputed)', 'avg(D_51_imputed)', 'avg(R_3_imputed)', 'avg(D_52_imputed)', 'avg(P_3_imputed)', 'avg(D_53_imputed)', 'avg(S_5_imputed)', 'avg(S_6_imputed)', 'avg(D_54_imputed)', 'avg(R_4_imputed)', 'avg(B_12_imputed)', 'avg(S_8_imputed)', 'avg(D_56_imputed)', 'avg(R_5_imputed)', 'avg(S_9_imputed)', 'avg(D_59_imputed)', 'avg(D_60_imputed)', 'avg(B_15_imputed)', 'avg(S_11_imputed)', 'avg(D_62_imputed)', 'avg(D_65_imputed)', 'avg(B_17_imputed)', 'avg(B_19_imputed)', 'avg(S_12_imputed)', 'avg(R_6_imputed)', 'avg(S_13_imputed)', 'avg(B_21_imputed)', 'avg(D_69_imputed)', 'avg(B_22_imputed

### Join target labels to the dataset on customerID

In [80]:
data_for_model = data_per_cust.join(train_labels, on='customer_ID')
# data_for_model = data_for_model.drop('S_2')
data_for_model.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['customer_ID', 'avg(P_2_imputed)', 'avg(D_39_imputed)', 'avg(B_1_imputed)', 'avg(B_2_imputed)', 'avg(R_1_imputed)', 'avg(S_3_imputed)', 'avg(D_41_imputed)', 'avg(B_3_imputed)', 'avg(D_43_imputed)', 'avg(D_44_imputed)', 'avg(B_4_imputed)', 'avg(D_45_imputed)', 'avg(B_5_imputed)', 'avg(R_2_imputed)', 'avg(D_46_imputed)', 'avg(D_47_imputed)', 'avg(D_48_imputed)', 'avg(B_6_imputed)', 'avg(B_8_imputed)', 'avg(D_50_imputed)', 'avg(D_51_imputed)', 'avg(R_3_imputed)', 'avg(D_52_imputed)', 'avg(P_3_imputed)', 'avg(D_53_imputed)', 'avg(S_5_imputed)', 'avg(S_6_imputed)', 'avg(D_54_imputed)', 'avg(R_4_imputed)', 'avg(B_12_imputed)', 'avg(S_8_imputed)', 'avg(D_56_imputed)', 'avg(R_5_imputed)', 'avg(S_9_imputed)', 'avg(D_59_imputed)', 'avg(D_60_imputed)', 'avg(B_15_imputed)', 'avg(S_11_imputed)', 'avg(D_62_imputed)', 'avg(D_65_imputed)', 'avg(B_17_imputed)', 'avg(B_19_imputed)', 'avg(S_12_imputed)', 'avg(R_6_imputed)', 'avg(S_13_imputed)', 'avg(B_21_imputed)', 'avg(D_69_imputed)', 'avg(B_22_imputed

### Get a list of all input column names to apply pipeline --> remove target

In [81]:
# Get all input column names to apply pipeline --> remove target
input_cols = data_per_cust.columns
input_cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['customer_ID', 'avg(P_2_imputed)', 'avg(D_39_imputed)', 'avg(B_1_imputed)', 'avg(B_2_imputed)', 'avg(R_1_imputed)', 'avg(S_3_imputed)', 'avg(D_41_imputed)', 'avg(B_3_imputed)', 'avg(D_43_imputed)', 'avg(D_44_imputed)', 'avg(B_4_imputed)', 'avg(D_45_imputed)', 'avg(B_5_imputed)', 'avg(R_2_imputed)', 'avg(D_46_imputed)', 'avg(D_47_imputed)', 'avg(D_48_imputed)', 'avg(B_6_imputed)', 'avg(B_8_imputed)', 'avg(D_50_imputed)', 'avg(D_51_imputed)', 'avg(R_3_imputed)', 'avg(D_52_imputed)', 'avg(P_3_imputed)', 'avg(D_53_imputed)', 'avg(S_5_imputed)', 'avg(S_6_imputed)', 'avg(D_54_imputed)', 'avg(R_4_imputed)', 'avg(B_12_imputed)', 'avg(S_8_imputed)', 'avg(D_56_imputed)', 'avg(R_5_imputed)', 'avg(S_9_imputed)', 'avg(D_59_imputed)', 'avg(D_60_imputed)', 'avg(B_15_imputed)', 'avg(S_11_imputed)', 'avg(D_62_imputed)', 'avg(D_65_imputed)', 'avg(B_17_imputed)', 'avg(B_19_imputed)', 'avg(S_12_imputed)', 'avg(R_6_imputed)', 'avg(S_13_imputed)', 'avg(B_21_imputed)', 'avg(D_69_imputed)', 'avg(B_22_imputed

In [82]:
input_cols.remove('customer_ID')
input_cols

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['avg(P_2_imputed)', 'avg(D_39_imputed)', 'avg(B_1_imputed)', 'avg(B_2_imputed)', 'avg(R_1_imputed)', 'avg(S_3_imputed)', 'avg(D_41_imputed)', 'avg(B_3_imputed)', 'avg(D_43_imputed)', 'avg(D_44_imputed)', 'avg(B_4_imputed)', 'avg(D_45_imputed)', 'avg(B_5_imputed)', 'avg(R_2_imputed)', 'avg(D_46_imputed)', 'avg(D_47_imputed)', 'avg(D_48_imputed)', 'avg(B_6_imputed)', 'avg(B_8_imputed)', 'avg(D_50_imputed)', 'avg(D_51_imputed)', 'avg(R_3_imputed)', 'avg(D_52_imputed)', 'avg(P_3_imputed)', 'avg(D_53_imputed)', 'avg(S_5_imputed)', 'avg(S_6_imputed)', 'avg(D_54_imputed)', 'avg(R_4_imputed)', 'avg(B_12_imputed)', 'avg(S_8_imputed)', 'avg(D_56_imputed)', 'avg(R_5_imputed)', 'avg(S_9_imputed)', 'avg(D_59_imputed)', 'avg(D_60_imputed)', 'avg(B_15_imputed)', 'avg(S_11_imputed)', 'avg(D_62_imputed)', 'avg(D_65_imputed)', 'avg(B_17_imputed)', 'avg(B_19_imputed)', 'avg(S_12_imputed)', 'avg(R_6_imputed)', 'avg(S_13_imputed)', 'avg(B_21_imputed)', 'avg(D_69_imputed)', 'avg(B_22_imputed)', 'avg(D_70_i

### Apply VectorAssembler, StandardScaler, PCA and Logistic Regression inside the Pipeline. Use CrossValidator for cross validation

In [84]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

transformer = VectorAssembler(inputCols=input_cols,
                              outputCol="features")

standardizer = StandardScaler(withMean=True, withStd=True,
                              inputCol='features',
                              outputCol='std_features')

pca = PCA(k=5, inputCol="std_features", outputCol="pca")

# default for maxIters is 100
lr = LogisticRegression(featuresCol='pca', labelCol='target')

pipeline = Pipeline(stages=[transformer, standardizer, pca, lr])

paramGrid = ParamGridBuilder()\
    .addGrid(lr.regParam, [0.1, 0.01]) \
    .addGrid(lr.fitIntercept, [False, True])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# # tvs = TrainValidationSplit(estimator=lr,
# #                            estimatorParamMaps=paramGrid,
# #                            evaluator=RegressionEvaluator(),
# #                            # 80% of the data will be used for training, 20% for validation.
# #                            trainRatio=0.8)

cv = CrossValidator(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol="target"),
                           # 80% of the data will be used for training, 20% for validation.
                        numFolds=2
                     #trainRatio=0.8
                   )



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Train-Test split

In [85]:
train, test = data_for_model.randomSplit([0.7, 0.3])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Fit the model on train batch

In [86]:
model = cv.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-86:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2291



https://stackoverflow.com/questions/58910023/keyerror-when-training-a-model-with-pyspark-ml-on-aws-emr-with-data-from-s3-buck
 The above error : "Exception in thread cell_monitor-65: KeyError: 1995"
 kept appearing every time I tried Crossvalidator or TrainValidationSplit. When they were removed, Pipeline.fit worked without errors. After multiple tries and lot of research, I found that the model still runs and produces the result.
In the above StackOverFlow discussion, it was mentioned that this error is related to the display of the progress bar and not the model itself. The model does seem to run fine and the cross validation has improved the accuracy from 0.85 to 0.87

In [87]:
# model = pipeline.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Make predictions on the test batch

In [92]:
prediction = model.transform(test)
score = prediction.select(['target', 'prediction'])
score.show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+
|target|prediction|
+------+----------+
|     0|       0.0|
|     0|       0.0|
|     1|       1.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     0|       0.0|
|     1|       1.0|
|     0|       0.0|
|     1|       0.0|
+------+----------+
only showing top 10 rows

### Calculate Accuracy and AUC metrics

In [89]:
acc = score.rdd.map(lambda x: x[0] == x[1]).sum() / score.count()
acc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8713367311479208

In [97]:
BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='target').evaluate(prediction)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8091731324914267

Since we did not set the BinaryClassificationEvaluatorMetricType in the above cell, the default value is AUC.
Hence, the metric that was calculated was :
- AUC = 0.8091

In [98]:
# from pyspark.mllib.evaluation import BinaryClassificationMetrics

# metrics = BinaryClassificationMetrics(score)
# auc = metrics.areaUnderROC
# apr = metrics.areaUnderPR
# auc, apr

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Time to run the model on the given test data

In [91]:
test_prediction = model.transform(data_per_cust_test)
test_prediction.select(['prediction']).show(n=10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
+----------+
only showing top 10 rows

The test predictions were made to create the prediction file to upload to kaggle for the Amex_pred challenge.